Mover las imagenes

In [3]:
import os
import shutil

# Rutas de las carpetas
carpeta_A = "/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/images"
carpeta_B = "/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Gatos"

# Extensiones de imágenes a mover
extensiones = (".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp")

contador = 0
for raiz, carpetas, archivos in os.walk(carpeta_A):
    for archivo in archivos:
        if archivo.lower().endswith(extensiones):
            origen = os.path.join(raiz, archivo)
            destino = os.path.join(carpeta_B, archivo)

            if os.path.exists(destino):
                nombre, ext = os.path.splitext(archivo)
                i = 1
                while os.path.exists(destino):
                    nuevo_nombre = f"{nombre}_{i}{ext}"
                    destino = os.path.join(carpeta_B, nuevo_nombre)
                    i += 1

            shutil.move(origen, destino)
            contador += 1

            # Solo mostrar mensaje cada 100 imágenes
            if contador % 100 == 0:
                print(f"Movidas {contador} imágenes...", end="\r")

print(f"Se movieron {contador} imágenes correctamente.")


Se movieron 126607 imágenes correctamente.


Eliminar algunas imagenes de forma uniforme

In [4]:
import os
import math

# Configuracion
carpeta = "/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Gatos"
total_deseado = 22000
extensiones = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

# Lista de imagenes
imagenes = [os.path.join(carpeta, f) for f in os.listdir(carpeta)
             if f.lower().endswith(extensiones)]
imagenes.sort()  

total_actual = len(imagenes)
print(f"Total actual: {total_actual}")

# Calcular Imagenes
if total_actual <= total_deseado:
    print("Ya tienes menos o igual a 20k imágenes, no se elimina nada.")
else:
    paso = total_actual / total_deseado
    print(f"Se conservará 1 de cada {paso:.2f} imágenes aproximadamente.")

    # --- Seleccionar imagenes a conservar
    indices_conservar = {math.floor(i * paso) for i in range(total_deseado)}
    
    # Eliminar el resto
    eliminadas = 0
    for idx, ruta in enumerate(imagenes):
        if idx not in indices_conservar:
            os.remove(ruta)
            eliminadas += 1
            if eliminadas % 1000 == 0:
                print(f"Eliminadas {eliminadas} imágenes...", end="\r")

    print(f"\n Eliminadas {eliminadas} imágenes. Quedan {total_deseado} en total.")


Total actual: 126607
Se conservará 1 de cada 5.75 imágenes aproximadamente.
🗑️  Eliminadas 104000 imágenes...
 Eliminadas 104607 imágenes. Quedan 22000 en total.


Extraer imagenes

In [2]:
import joblib
import numpy as np
from PIL import Image
import os

# Nombre del archivo .sav que descargaste
archivo_sav = '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/turtles128.sav'

# Carpeta donde quieres guardar las imágenes
carpeta_salida = '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Tortuga_aux'

#Crear la carpeta de salida si no existe
if not os.path.exists(carpeta_salida):
    os.makedirs(carpeta_salida)
    print(f"Carpeta '{carpeta_salida}' creada.")

#Cargar el archivo .sav
print(f"Cargando '{archivo_sav}'... (Esto puede tardar un momento)")
lista_de_imagenes = joblib.load(archivo_sav)
print(f"¡Archivo cargado! Se encontraron {len(lista_de_imagenes)} imágenes.")

#Convertir la lista a un array de NumPy
dataset_array = np.array(lista_de_imagenes)

print(f"Guardando imágenes en '{carpeta_salida}'...")
for i, img_array in enumerate(dataset_array):
    

    #Escalamos de [0, 1] a [0, 255] y convertimos a entero
    img_array_uint8 = (img_array * 255.0).astype(np.uint8)

    # Crear un objeto de imagen desde el array de píxeles
    img = Image.fromarray(img_array_uint8)
    
    # Definir el nombre del archivo (ej: tortuga_00001.png)
    nombre_archivo = f'tortuga_{str(i).zfill(5)}.png'
    ruta_archivo = os.path.join(carpeta_salida, nombre_archivo)
    
    # Guardar la imagen
    img.save(ruta_archivo)

print(f"¡Listo! Se guardaron {len(dataset_array)} imágenes en la carpeta '{carpeta_salida}'.")

Cargando '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/turtles128.sav'... (Esto puede tardar un momento)
¡Archivo cargado! Se encontraron 17456 imágenes.
Guardando imágenes en '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Tortuga_aux'...
¡Listo! Se guardaron 17456 imágenes en la carpeta '/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Tortuga_aux'.


Data augmentation

In [3]:
import os
import random
from PIL import Image, ImageEnhance, ImageOps
from tqdm import tqdm

#Configuración
carpeta = "/home/yisus/Escritorio/Datasets/DatasetsAnimales/Datasets AI/Tortuga"  # Carpeta con las imágenes originales
num_a_generar = 2000  # Cuántas imágenes nuevas quieres generar
extensiones = (".jpg", ".jpeg", ".png")

#Función para aplicar transformaciones
def transformar_imagen(img):
    # Elegir una o varias transformaciones al azar
    if random.random() < 0.5:
        img = img.transpose(Image.FLIP_LEFT_RIGHT)
    if random.random() < 0.3:
        img = img.transpose(Image.FLIP_TOP_BOTTOM)
    if random.random() < 0.3:
        angulo = random.randint(-30, 30)
        img = img.rotate(angulo)
    if random.random() < 0.4:
        factor_brillo = random.uniform(0.7, 1.3)
        img = ImageEnhance.Brightness(img).enhance(factor_brillo)
    if random.random() < 0.4:
        factor_contraste = random.uniform(0.8, 1.2)
        img = ImageEnhance.Contrast(img).enhance(factor_contraste)
    if random.random() < 0.3:
        img = ImageOps.autocontrast(img)
    return img

# btener imágenes originales
imagenes = [f for f in os.listdir(carpeta) if f.lower().endswith(extensiones)]
print(f"📸 Imágenes originales: {len(imagenes)}")

# Generar nuevas imágenes
for i in tqdm(range(num_a_generar), desc="Generando imágenes"):
    archivo_original = random.choice(imagenes)
    ruta_original = os.path.join(carpeta, archivo_original)

    try:
        img = Image.open(ruta_original)
        img = transformar_imagen(img)

        nombre, ext = os.path.splitext(archivo_original)
        nuevo_nombre = f"{nombre}_aug{i}{ext}"
        ruta_nueva = os.path.join(carpeta, nuevo_nombre)

        img.save(ruta_nueva)
    except Exception as e:
        print(f"Error con {archivo_original}: {e}")

print(f"\nSe generaron {num_a_generar} imágenes nuevas en la misma carpeta.")


📸 Imágenes originales: 19864


Generando imágenes: 100%|██████████| 2000/2000 [02:31<00:00, 13.20it/s]


Se generaron 2000 imágenes nuevas en la misma carpeta.
